In [1]:
import random
import math

#
# Shorthand:
#   "pd_" as a variable prefix means "partial derivative"
#   "d_" as a variable prefix means "derivative"
#   "_wrt_" is shorthand for "with respect to"
#   "w_ho" and "w_ih" are the index of weights from hidden to output layer neurons and input to hidden layer neurons respectively
#
# Comment references:
#
# [1] Wikipedia article on Backpropagation
#   http://en.wikipedia.org/wiki/Backpropagation#Finding_the_derivative_of_the_error
# [2] Neural Networks for Machine Learning course on Coursera by Geoffrey Hinton
#   https://class.coursera.org/neuralnets-2012-001/lecture/39
# [3] The Back Propagation Algorithm
#   https://www4.rgu.ac.uk/files/chapter3%20-%20bp.pdf



In [2]:
class NeuralNetwork:
    LEARNING_RATE = 0.5

    def __init__(self, num_inputs, num_hidden, num_outputs, hidden_layer_weights = None, hidden_layer_bias = None, output_layer_weights = None, output_layer_bias = None):
        self.num_inputs = num_inputs

        self.hidden_layer = NeuronLayer(num_hidden, hidden_layer_bias)
        self.output_layer = NeuronLayer(num_outputs, output_layer_bias)

        self.init_weights_from_inputs_to_hidden_layer_neurons(hidden_layer_weights)
        self.init_weights_from_hidden_layer_neurons_to_output_layer_neurons(output_layer_weights)

    def init_weights_from_inputs_to_hidden_layer_neurons(self, hidden_layer_weights):
        weight_num = 0
        for h in range(len(self.hidden_layer.neurons)):
            for i in range(self.num_inputs):
                if not hidden_layer_weights:
                    self.hidden_layer.neurons[h].weights.append(random.random())
                else:
                    self.hidden_layer.neurons[h].weights.append(hidden_layer_weights[weight_num])
                weight_num += 1

    def init_weights_from_hidden_layer_neurons_to_output_layer_neurons(self, output_layer_weights):
        weight_num = 0
        for o in range(len(self.output_layer.neurons)):
            for h in range(len(self.hidden_layer.neurons)):
                if not output_layer_weights:
                    self.output_layer.neurons[o].weights.append(random.random())
                else:
                    self.output_layer.neurons[o].weights.append(output_layer_weights[weight_num])
                weight_num += 1

    def inspect(self):
        print('------')
        print('* Inputs: {}'.format(self.num_inputs))
        print('------')
        print('Hidden Layer')
        self.hidden_layer.inspect()
        print('------')
        print('* Output Layer')
        self.output_layer.inspect()
        print('------')

    def feed_forward(self, inputs):
        hidden_layer_outputs = self.hidden_layer.feed_forward(inputs)
        return self.output_layer.feed_forward(hidden_layer_outputs)

    # Uses online learning, ie updating the weights after each training case
    def train(self, training_inputs, training_outputs):
        self.feed_forward(training_inputs)

        # 1. Output neuron deltas
        pd_errors_wrt_output_neuron_total_net_input = [0] * len(self.output_layer.neurons)
        for o in range(len(self.output_layer.neurons)):

            # ∂E/∂zⱼ
            pd_errors_wrt_output_neuron_total_net_input[o] = self.output_layer.neurons[o].calculate_pd_error_wrt_total_net_input(training_outputs[o])

        # 2. Hidden neuron deltas
        pd_errors_wrt_hidden_neuron_total_net_input = [0] * len(self.hidden_layer.neurons)
        for h in range(len(self.hidden_layer.neurons)):

            # We need to calculate the derivative of the error with respect to the output of each hidden layer neuron
            # dE/dyⱼ = Σ ∂E/∂zⱼ * ∂z/∂yⱼ = Σ ∂E/∂zⱼ * wᵢⱼ
            d_error_wrt_hidden_neuron_output = 0
            for o in range(len(self.output_layer.neurons)):
                d_error_wrt_hidden_neuron_output += pd_errors_wrt_output_neuron_total_net_input[o] * self.output_layer.neurons[o].weights[h]

            # ∂E/∂zⱼ = dE/dyⱼ * ∂zⱼ/∂
            pd_errors_wrt_hidden_neuron_total_net_input[h] = d_error_wrt_hidden_neuron_output * self.hidden_layer.neurons[h].calculate_pd_total_net_input_wrt_input()

        # 3. Update output neuron weights
        for o in range(len(self.output_layer.neurons)):
            for w_ho in range(len(self.output_layer.neurons[o].weights)):

                # ∂Eⱼ/∂wᵢⱼ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢⱼ
                pd_error_wrt_weight = pd_errors_wrt_output_neuron_total_net_input[o] * self.output_layer.neurons[o].calculate_pd_total_net_input_wrt_weight(w_ho)

                # Δw = α * ∂Eⱼ/∂wᵢ
                self.output_layer.neurons[o].weights[w_ho] -= self.LEARNING_RATE * pd_error_wrt_weight

        # 4. Update hidden neuron weights
        for h in range(len(self.hidden_layer.neurons)):
            for w_ih in range(len(self.hidden_layer.neurons[h].weights)):

                # ∂Eⱼ/∂wᵢ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢ
                pd_error_wrt_weight = pd_errors_wrt_hidden_neuron_total_net_input[h] * self.hidden_layer.neurons[h].calculate_pd_total_net_input_wrt_weight(w_ih)

                # Δw = α * ∂Eⱼ/∂wᵢ
                self.hidden_layer.neurons[h].weights[w_ih] -= self.LEARNING_RATE * pd_error_wrt_weight

    def calculate_total_error(self, training_sets):
        total_error = 0
        for t in range(len(training_sets)):
            training_inputs, training_outputs = training_sets[t]
            self.feed_forward(training_inputs)
            for o in range(len(training_outputs)):
                total_error += self.output_layer.neurons[o].calculate_error(training_outputs[o])
        return total_error



In [3]:
class NeuronLayer:
    def __init__(self, num_neurons, bias):

        # Every neuron in a layer shares the same bias
        self.bias = bias if bias else random.random()

        self.neurons = []
        for i in range(num_neurons):
            self.neurons.append(Neuron(self.bias))

    def inspect(self):
        print('Neurons:', len(self.neurons))
        for n in range(len(self.neurons)):
            print(' Neuron', n)
            for w in range(len(self.neurons[n].weights)):
                print('  Weight:', self.neurons[n].weights[w])
            print('  Bias:', self.bias)

    def feed_forward(self, inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs

    def get_outputs(self):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.output)
        return outputs



In [4]:
class Neuron:
    def __init__(self, bias):
        self.bias = bias
        self.weights = []

    def calculate_output(self, inputs):
        self.inputs = inputs
        self.output = self.squash(self.calculate_total_net_input())
        return self.output

    def calculate_total_net_input(self):
        total = 0
        for i in range(len(self.inputs)):
            total += self.inputs[i] * self.weights[i]
        return total + self.bias

    # Apply the logistic function to squash the output of the neuron
    # The result is sometimes referred to as 'net' [2] or 'net' [1]
    def squash(self, total_net_input):
        return 1 / (1 + math.exp(-total_net_input))

    # Determine how much the neuron's total input has to change to move closer to the expected output
    #
    # Now that we have the partial derivative of the error with respect to the output (∂E/∂yⱼ) and
    # the derivative of the output with respect to the total net input (dyⱼ/dzⱼ) we can calculate
    # the partial derivative of the error with respect to the total net input.
    # This value is also known as the delta (δ) [1]
    # δ = ∂E/∂zⱼ = ∂E/∂yⱼ * dyⱼ/dzⱼ
    #
    def calculate_pd_error_wrt_total_net_input(self, target_output):
        return self.calculate_pd_error_wrt_output(target_output) * self.calculate_pd_total_net_input_wrt_input();

    # The error for each neuron is calculated by the Mean Square Error method:
    def calculate_error(self, target_output):
        return 0.5 * (target_output - self.output) ** 2

    # The partial derivate of the error with respect to actual output then is calculated by:
    # = 2 * 0.5 * (target output - actual output) ^ (2 - 1) * -1
    # = -(target output - actual output)
    #
    # The Wikipedia article on backpropagation [1] simplifies to the following, but most other learning material does not [2]
    # = actual output - target output
    #
    # Alternative, you can use (target - output), but then need to add it during backpropagation [3]
    #
    # Note that the actual output of the output neuron is often written as yⱼ and target output as tⱼ so:
    # = ∂E/∂yⱼ = -(tⱼ - yⱼ)
    def calculate_pd_error_wrt_output(self, target_output):
        return -(target_output - self.output)

    # The total net input into the neuron is squashed using logistic function to calculate the neuron's output:
    # yⱼ = φ = 1 / (1 + e^(-zⱼ))
    # Note that where ⱼ represents the output of the neurons in whatever layer we're looking at and ᵢ represents the layer below it
    #
    # The derivative (not partial derivative since there is only one variable) of the output then is:
    # dyⱼ/dzⱼ = yⱼ * (1 - yⱼ)
    def calculate_pd_total_net_input_wrt_input(self):
        return self.output * (1 - self.output)

    # The total net input is the weighted sum of all the inputs to the neuron and their respective weights:
    # = zⱼ = netⱼ = x₁w₁ + x₂w₂ ...
    #
    # The partial derivative of the total net input with respective to a given weight (with everything else held constant) then is:
    # = ∂zⱼ/∂wᵢ = some constant + 1 * xᵢw₁^(1-0) + some constant ... = xᵢ
    def calculate_pd_total_net_input_wrt_weight(self, index):
        return self.inputs[index]




In [5]:
nn = NeuralNetwork(2, 2, 2, hidden_layer_weights=[0.15, 0.2, 0.25, 0.3], hidden_layer_bias=0.35, output_layer_weights=[0.4, 0.45, 0.5, 0.55], output_layer_bias=0.6)
for i in range(10000):
    nn.train([0.05, 0.1], [0.01, 0.99])
    print(i, round(nn.calculate_total_error([[[0.05, 0.1], [0.01, 0.99]]]), 9))



0 0.291027774
1 0.283547133
2 0.275943289
3 0.268232761
4 0.260434393
5 0.252569176
6 0.244659999
7 0.236731316
8 0.228808741
9 0.220918592
10 0.213087389
11 0.205341328
12 0.197705769
13 0.190204742
14 0.182860503
15 0.175693166
16 0.168720403
17 0.16195725
18 0.155415989
19 0.149106135
20 0.14303449
21 0.137205276
22 0.131620316
23 0.126279262
24 0.121179847
25 0.116318143
26 0.111688831
27 0.107285459
28 0.103100677
29 0.099126464
30 0.095354325
31 0.091775464
32 0.088380932
33 0.085161757
34 0.082109043
35 0.079214055
36 0.076468284
37 0.073863495
38 0.071391768
39 0.069045516
40 0.066817506
41 0.064700863
42 0.062689072
43 0.060775976
44 0.058955766
45 0.057222975
46 0.05557246
47 0.053999393
48 0.052499243
49 0.051067764
50 0.049700977
51 0.048395156
52 0.047146814
53 0.045952686
54 0.044809717
55 0.043715048
56 0.042666004
57 0.041660079
58 0.040694928
59 0.039768356
60 0.038878306
61 0.03802285
62 0.037200182
63 0.036408606
64 0.035646534
65 0.034912472
66 0.03420502
67 0.03352

634 0.001984051
635 0.001980134
636 0.001976231
637 0.001972341
638 0.001968465
639 0.001964602
640 0.001960752
641 0.001956916
642 0.001953093
643 0.001949283
644 0.001945486
645 0.001941703
646 0.001937932
647 0.001934174
648 0.001930429
649 0.001926696
650 0.001922976
651 0.001919269
652 0.001915575
653 0.001911893
654 0.001908223
655 0.001904566
656 0.001900921
657 0.001897288
658 0.001893668
659 0.00189006
660 0.001886463
661 0.001882879
662 0.001879307
663 0.001875747
664 0.001872198
665 0.001868662
666 0.001865137
667 0.001861623
668 0.001858122
669 0.001854632
670 0.001851153
671 0.001847686
672 0.00184423
673 0.001840786
674 0.001837353
675 0.001833931
676 0.00183052
677 0.001827121
678 0.001823732
679 0.001820355
680 0.001816989
681 0.001813633
682 0.001810289
683 0.001806955
684 0.001803632
685 0.001800319
686 0.001797018
687 0.001793727
688 0.001790446
689 0.001787176
690 0.001783917
691 0.001780668
692 0.001777429
693 0.001774201
694 0.001770983
695 0.001767775
696 0.00176

1189 0.000889542
1190 0.000888568
1191 0.000887597
1192 0.000886627
1193 0.000885658
1194 0.000884692
1195 0.000883728
1196 0.000882765
1197 0.000881804
1198 0.000880844
1199 0.000879887
1200 0.000878931
1201 0.000877977
1202 0.000877025
1203 0.000876075
1204 0.000875126
1205 0.000874179
1206 0.000873234
1207 0.000872291
1208 0.000871349
1209 0.000870409
1210 0.000869471
1211 0.000868535
1212 0.0008676
1213 0.000866667
1214 0.000865736
1215 0.000864806
1216 0.000863878
1217 0.000862952
1218 0.000862027
1219 0.000861105
1220 0.000860183
1221 0.000859264
1222 0.000858346
1223 0.00085743
1224 0.000856516
1225 0.000855603
1226 0.000854692
1227 0.000853782
1228 0.000852874
1229 0.000851968
1230 0.000851064
1231 0.000850161
1232 0.000849259
1233 0.00084836
1234 0.000847462
1235 0.000846565
1236 0.00084567
1237 0.000844777
1238 0.000843885
1239 0.000842995
1240 0.000842107
1241 0.00084122
1242 0.000840335
1243 0.000839451
1244 0.000838569
1245 0.000837689
1246 0.00083681
1247 0.000835933
1248

1783 0.000519776
1784 0.000519382
1785 0.000518989
1786 0.000518596
1787 0.000518203
1788 0.000517811
1789 0.00051742
1790 0.000517029
1791 0.000516638
1792 0.000516248
1793 0.000515859
1794 0.00051547
1795 0.000515081
1796 0.000514693
1797 0.000514305
1798 0.000513918
1799 0.000513531
1800 0.000513145
1801 0.00051276
1802 0.000512374
1803 0.00051199
1804 0.000511605
1805 0.000511222
1806 0.000510838
1807 0.000510455
1808 0.000510073
1809 0.000509691
1810 0.00050931
1811 0.000508929
1812 0.000508549
1813 0.000508169
1814 0.000507789
1815 0.00050741
1816 0.000507031
1817 0.000506653
1818 0.000506276
1819 0.000505899
1820 0.000505522
1821 0.000505146
1822 0.00050477
1823 0.000504394
1824 0.00050402
1825 0.000503645
1826 0.000503271
1827 0.000502898
1828 0.000502525
1829 0.000502152
1830 0.00050178
1831 0.000501409
1832 0.000501037
1833 0.000500667
1834 0.000500296
1835 0.000499927
1836 0.000499557
1837 0.000499188
1838 0.00049882
1839 0.000498452
1840 0.000498084
1841 0.000497717
1842 0.

2379 0.000349629
2380 0.000349423
2381 0.000349218
2382 0.000349013
2383 0.000348808
2384 0.000348603
2385 0.000348398
2386 0.000348194
2387 0.00034799
2388 0.000347786
2389 0.000347582
2390 0.000347378
2391 0.000347175
2392 0.000346972
2393 0.000346769
2394 0.000346566
2395 0.000346363
2396 0.000346161
2397 0.000345958
2398 0.000345756
2399 0.000345554
2400 0.000345353
2401 0.000345151
2402 0.00034495
2403 0.000344749
2404 0.000344548
2405 0.000344347
2406 0.000344146
2407 0.000343946
2408 0.000343746
2409 0.000343546
2410 0.000343346
2411 0.000343146
2412 0.000342947
2413 0.000342748
2414 0.000342549
2415 0.00034235
2416 0.000342151
2417 0.000341953
2418 0.000341754
2419 0.000341556
2420 0.000341358
2421 0.000341161
2422 0.000340963
2423 0.000340766
2424 0.000340568
2425 0.000340371
2426 0.000340175
2427 0.000339978
2428 0.000339781
2429 0.000339585
2430 0.000339389
2431 0.000339193
2432 0.000338997
2433 0.000338802
2434 0.000338607
2435 0.000338411
2436 0.000338216
2437 0.000338022


...

7171 6.4038e-05
7172 6.4023e-05
7173 6.4008e-05
7174 6.3992e-05
7175 6.3977e-05
7176 6.3961e-05
7177 6.3946e-05
7178 6.3931e-05
7179 6.3915e-05
7180 6.39e-05
7181 6.3885e-05
7182 6.3869e-05
7183 6.3854e-05
7184 6.3839e-05
7185 6.3823e-05
7186 6.3808e-05
7187 6.3793e-05
7188 6.3777e-05
7189 6.3762e-05
7190 6.3747e-05
7191 6.3731e-05
7192 6.3716e-05
7193 6.3701e-05
7194 6.3685e-05
7195 6.367e-05
7196 6.3655e-05
7197 6.364e-05
7198 6.3624e-05
7199 6.3609e-05
7200 6.3594e-05
7201 6.3578e-05
7202 6.3563e-05
7203 6.3548e-05
7204 6.3533e-05
7205 6.3517e-05
7206 6.3502e-05
7207 6.3487e-05
7208 6.3472e-05
7209 6.3457e-05
7210 6.3441e-05
7211 6.3426e-05
7212 6.3411e-05
7213 6.3396e-05
7214 6.3381e-05
7215 6.3365e-05
7216 6.335e-05
7217 6.3335e-05
7218 6.332e-05
7219 6.3305e-05
7220 6.329e-05
7221 6.3274e-05
7222 6.3259e-05
7223 6.3244e-05
7224 6.3229e-05
7225 6.3214e-05
7226 6.3199e-05
7227 6.3184e-05
7228 6.3169e-05
7229 6.3153e-05
7230 6.3138e-05
7231 6.3123e-05
7232 6.3108e-05
7233 6.3093e-05

7749 5.5942e-05
7750 5.593e-05
7751 5.5917e-05
7752 5.5904e-05
7753 5.5891e-05
7754 5.5879e-05
7755 5.5866e-05
7756 5.5853e-05
7757 5.5841e-05
7758 5.5828e-05
7759 5.5815e-05
7760 5.5803e-05
7761 5.579e-05
7762 5.5777e-05
7763 5.5765e-05
7764 5.5752e-05
7765 5.5739e-05
7766 5.5727e-05
7767 5.5714e-05
7768 5.5701e-05
7769 5.5689e-05
7770 5.5676e-05
7771 5.5663e-05
7772 5.5651e-05
7773 5.5638e-05
7774 5.5626e-05
7775 5.5613e-05
7776 5.56e-05
7777 5.5588e-05
7778 5.5575e-05
7779 5.5563e-05
7780 5.555e-05
7781 5.5537e-05
7782 5.5525e-05
7783 5.5512e-05
7784 5.55e-05
7785 5.5487e-05
7786 5.5474e-05
7787 5.5462e-05
7788 5.5449e-05
7789 5.5437e-05
7790 5.5424e-05
7791 5.5412e-05
7792 5.5399e-05
7793 5.5387e-05
7794 5.5374e-05
7795 5.5361e-05
7796 5.5349e-05
7797 5.5336e-05
7798 5.5324e-05
7799 5.5311e-05
7800 5.5299e-05
7801 5.5286e-05
7802 5.5274e-05
7803 5.5261e-05
7804 5.5249e-05
7805 5.5236e-05
7806 5.5224e-05
7807 5.5211e-05
7808 5.5199e-05
7809 5.5186e-05
7810 5.5174e-05
7811 5.5161e-05

8311 4.939e-05
8312 4.938e-05
8313 4.9369e-05
8314 4.9358e-05
8315 4.9348e-05
8316 4.9337e-05
8317 4.9326e-05
8318 4.9316e-05
8319 4.9305e-05
8320 4.9294e-05
8321 4.9284e-05
8322 4.9273e-05
8323 4.9262e-05
8324 4.9252e-05
8325 4.9241e-05
8326 4.923e-05
8327 4.922e-05
8328 4.9209e-05
8329 4.9199e-05
8330 4.9188e-05
8331 4.9177e-05
8332 4.9167e-05
8333 4.9156e-05
8334 4.9145e-05
8335 4.9135e-05
8336 4.9124e-05
8337 4.9114e-05
8338 4.9103e-05
8339 4.9092e-05
8340 4.9082e-05
8341 4.9071e-05
8342 4.9061e-05
8343 4.905e-05
8344 4.904e-05
8345 4.9029e-05
8346 4.9018e-05
8347 4.9008e-05
8348 4.8997e-05
8349 4.8987e-05
8350 4.8976e-05
8351 4.8966e-05
8352 4.8955e-05
8353 4.8945e-05
8354 4.8934e-05
8355 4.8923e-05
8356 4.8913e-05
8357 4.8902e-05
8358 4.8892e-05
8359 4.8881e-05
8360 4.8871e-05
8361 4.886e-05
8362 4.885e-05
8363 4.8839e-05
8364 4.8829e-05
8365 4.8818e-05
8366 4.8808e-05
8367 4.8797e-05
8368 4.8787e-05
8369 4.8776e-05
8370 4.8766e-05
8371 4.8755e-05
8372 4.8745e-05
8373 4.8734e-05


8905 4.3574e-05
8906 4.3565e-05
8907 4.3556e-05
8908 4.3547e-05
8909 4.3539e-05
8910 4.353e-05
8911 4.3521e-05
8912 4.3512e-05
8913 4.3503e-05
8914 4.3494e-05
8915 4.3485e-05
8916 4.3476e-05
8917 4.3467e-05
8918 4.3458e-05
8919 4.3449e-05
8920 4.344e-05
8921 4.3431e-05
8922 4.3422e-05
8923 4.3413e-05
8924 4.3404e-05
8925 4.3395e-05
8926 4.3387e-05
8927 4.3378e-05
8928 4.3369e-05
8929 4.336e-05
8930 4.3351e-05
8931 4.3342e-05
8932 4.3333e-05
8933 4.3324e-05
8934 4.3315e-05
8935 4.3306e-05
8936 4.3298e-05
8937 4.3289e-05
8938 4.328e-05
8939 4.3271e-05
8940 4.3262e-05
8941 4.3253e-05
8942 4.3244e-05
8943 4.3235e-05
8944 4.3226e-05
8945 4.3218e-05
8946 4.3209e-05
8947 4.32e-05
8948 4.3191e-05
8949 4.3182e-05
8950 4.3173e-05
8951 4.3164e-05
8952 4.3156e-05
8953 4.3147e-05
8954 4.3138e-05
8955 4.3129e-05
8956 4.312e-05
8957 4.3111e-05
8958 4.3102e-05
8959 4.3094e-05
8960 4.3085e-05
8961 4.3076e-05
8962 4.3067e-05
8963 4.3058e-05
8964 4.305e-05
8965 4.3041e-05
8966 4.3032e-05
8967 4.3023e-05


9463 3.894e-05
9464 3.8932e-05
9465 3.8925e-05
9466 3.8917e-05
9467 3.8909e-05
9468 3.8902e-05
9469 3.8894e-05
9470 3.8886e-05
9471 3.8879e-05
9472 3.8871e-05
9473 3.8863e-05
9474 3.8856e-05
9475 3.8848e-05
9476 3.884e-05
9477 3.8833e-05
9478 3.8825e-05
9479 3.8817e-05
9480 3.881e-05
9481 3.8802e-05
9482 3.8794e-05
9483 3.8787e-05
9484 3.8779e-05
9485 3.8772e-05
9486 3.8764e-05
9487 3.8756e-05
9488 3.8749e-05
9489 3.8741e-05
9490 3.8733e-05
9491 3.8726e-05
9492 3.8718e-05
9493 3.871e-05
9494 3.8703e-05
9495 3.8695e-05
9496 3.8688e-05
9497 3.868e-05
9498 3.8672e-05
9499 3.8665e-05
9500 3.8657e-05
9501 3.865e-05
9502 3.8642e-05
9503 3.8634e-05
9504 3.8627e-05
9505 3.8619e-05
9506 3.8612e-05
9507 3.8604e-05
9508 3.8596e-05
9509 3.8589e-05
9510 3.8581e-05
9511 3.8574e-05
9512 3.8566e-05
9513 3.8558e-05
9514 3.8551e-05
9515 3.8543e-05
9516 3.8536e-05
9517 3.8528e-05
9518 3.8521e-05
9519 3.8513e-05
9520 3.8505e-05
9521 3.8498e-05
9522 3.849e-05
9523 3.8483e-05
9524 3.8475e-05
9525 3.8468e-05

In [8]:
# XOR example:

training_sets = [
     [[0, 0], [0]],
     [[0, 1], [1]],
     [[1, 0], [1]],
     [[1, 1], [0]]
 ]

nn = NeuralNetwork(len(training_sets[0][0]), 5, len(training_sets[0][1]))
for i in range(10000):
    training_inputs, training_outputs = random.choice(training_sets)
    nn.train(training_inputs, training_outputs)
    print(i, nn.calculate_total_error(training_sets))

0 0.6748588717967754
1 0.6803198937086244
2 0.6616137637036362
3 0.6424323504169371
4 0.6230516392200767
5 0.6305030346123182
6 0.6377557142506562
7 0.6183571466817582
8 0.6260809760930718
9 0.6334167056935762
10 0.6140194001987437
11 0.6219139753779741
12 0.6026517766999511
13 0.5839599843201122
14 0.5930327500837334
15 0.6017681653254204
16 0.6101508711507271
17 0.5861497367071481
18 0.5949978264444169
19 0.6036412121570738
20 0.6119326917486911
21 0.5879232906219167
22 0.5968202155916036
23 0.6053315170454511
24 0.6135517723727161
25 0.5944919278852001
26 0.6031475552160628
27 0.6114507455287157
28 0.6194758480921901
29 0.6002442238572152
30 0.5816393875333605
31 0.5590563600316857
32 0.5435956931826889
33 0.5301535535148422
34 0.5157984731375771
35 0.506107982889327
36 0.51189116907091
37 0.5038697313745628
38 0.5088150775221202
39 0.5034813129110763
40 0.500685337689657
41 0.5002638673761525
42 0.500792349011137
43 0.5041386771355442
44 0.5004543687315866
45 0.5032071421026778
46 

513 0.49955180489460577
514 0.5014862176045699
515 0.4996306322607022
516 0.5001335688704881
517 0.4998643435749064
518 0.5027543207366317
519 0.5068027434608181
520 0.5014056233042556
521 0.4995700800630825
522 0.500670097911847
523 0.5035384085553646
524 0.500015958195853
525 0.5032138107406875
526 0.5001740601960959
527 0.5025798856097696
528 0.4999302092597977
529 0.49992048888193374
530 0.5000176328374086
531 0.5022354958695119
532 0.49965853090338425
533 0.5020875531876015
534 0.49978166347673886
535 0.5016379065177152
536 0.49956311690824773
537 0.5003163366433346
538 0.5030526533856512
539 0.5074221524801635
540 0.5130884576106196
541 0.5197454811906136
542 0.5271261691475286
543 0.5168278719368813
544 0.5252643211717721
545 0.5128812701861699
546 0.5045640401010719
547 0.5104749517205512
548 0.504476447271864
549 0.5008688631608275
550 0.49955721132200853
551 0.5006441071323691
552 0.49961646148488037
553 0.5001511446115117
554 0.49958423289081105
555 0.5017529409666238
556 0.

978 0.5097214972623774
979 0.5030453972581583
980 0.5002733406993715
981 0.49976664743511384
982 0.4999672754473945
983 0.499718736250679
984 0.5019852485477108
985 0.49981001688573445
986 0.5015888778524131
987 0.5049329090214145
988 0.5108419730465664
989 0.5181962252723478
990 0.5086973882972388
991 0.5032101019510765
992 0.5084459146895354
993 0.5030599730033559
994 0.5001925378985415
995 0.49974842676893727
996 0.5019542826959928
997 0.5063189347040412
998 0.5019744444944598
999 0.499848583246953
1000 0.5018477152242188
1001 0.4996294595951301
1002 0.5014717789365257
1003 0.49969993034313076
1004 0.5003087848547714
1005 0.49960716403737665
1006 0.5001728167572894
1007 0.5024731212047797
1008 0.5073706031433611
1009 0.5126441239837756
1010 0.5203058726731287
1011 0.5112738650901822
1012 0.5048740177791946
1013 0.5006552064096431
1014 0.4995858622003442
1015 0.5006656625238275
1016 0.4995640110174002
1017 0.5009994084241562
1018 0.5049362377044557
1019 0.5007015242054141
1020 0.5043

1499 0.5049128230264422
1500 0.5011820652784831
1501 0.5049908697839659
1502 0.5098410511160387
1503 0.5163794602846732
1504 0.5237951858330792
1505 0.514564697991624
1506 0.5217769094616864
1507 0.5296365064012586
1508 0.5172002183236637
1509 0.5095038370727352
1510 0.5159557804847972
1511 0.523296183201437
1512 0.5312235560155025
1513 0.520553136026471
1514 0.5278279372818531
1515 0.535587158797261
1516 0.5219257144052102
1517 0.5131112091718575
1518 0.5052271434037836
1519 0.500722900127854
1520 0.49943299848192024
1521 0.5007315987362195
1522 0.503758875374297
1523 0.5000816790138364
1524 0.50297128375218
1525 0.5076719860818724
1526 0.5019716520903748
1527 0.49978297785238773
1528 0.49951385500657275
1529 0.5016311983997584
1530 0.49951485555648933
1531 0.5015756398748248
1532 0.4995845524618278
1533 0.5011628533274389
1534 0.5041979888917182
1535 0.5003791551898991
1536 0.5028790870878462
1537 0.5079554801197105
1538 0.5146787497331231
1539 0.506482540533677
1540 0.51282525039808

1977 0.5044972781110518
1978 0.5100699897692504
1979 0.5156347742291438
1980 0.5080825537257141
1981 0.5146264542265104
1982 0.5073580988602842
1983 0.5124339128915606
1984 0.5049837528019692
1985 0.501059982773362
1986 0.4993270257746195
1987 0.501363298832145
1988 0.49941274733248386
1989 0.4994676027623398
1990 0.4996196962580024
1991 0.4993176669623467
1992 0.4995530891217992
1993 0.49948413546910336
1994 0.49959595842255994
1995 0.4994743394680905
1996 0.4993889600695827
1997 0.5009839522346814
1998 0.5040240968937351
1999 0.5001909132230888
2000 0.5035174858362296
2001 0.5002911847524311
2002 0.5037082257643748
2003 0.5089969765292963
2004 0.5143852654384079
2005 0.5072061504607219
2006 0.5023731775426904
2007 0.507103477580246
2008 0.5017321156943673
2009 0.4995285104231129
2010 0.4993438897585354
2011 0.4997685222869995
2012 0.4992420484747759
2013 0.49968543484601685
2014 0.49924822694865034
2015 0.5010301202332704
2016 0.5048701177569104
2017 0.5103285297286191
2018 0.5035875

...

7973 0.030829094620937542
7974 0.030843864070009795
7975 0.030879786859866705
7976 0.030817096317983034
7977 0.030766961432178955
7978 0.030779060138049652
7979 0.030808885223466517
7980 0.03085824700403534
7981 0.030758519722191985
7982 0.030802498941315506
7983 0.03086469081727635
7984 0.03094337325618332
7985 0.031035853379069375
7986 0.030905879821873587
7987 0.030760072849891174
7988 0.0306599641471191
7989 0.03059953083666437
7990 0.030571779988515006
7991 0.03056811440530375
7992 0.030577929313007908
7993 0.030602428215284395
7994 0.030535792294956718
7995 0.030555803984311174
7996 0.030496805103325624
7997 0.030528399680837537
7998 0.030563122461666287
7999 0.030484567187560156
8000 0.03051475547426862
8001 0.030580962734369166
8002 0.03066687031819123
8003 0.030539098477103573
8004 0.03062192025485305
8005 0.030500406346287402
8006 0.030410940493582107
8007 0.03035016203286086
8008 0.030315138954626
8009 0.03031717594910871
8010 0.03028717938622555
8011 0.0302714599486595
8012

8431 0.025202719768960137
8432 0.02521906079975094
8433 0.025185124729603506
8434 0.02516397786803362
8435 0.025154599935011623
8436 0.025140831715903793
8437 0.025142791668611553
8438 0.025114048024251927
8439 0.025119026909826817
8440 0.02508362203429688
8441 0.025086936642426918
8442 0.025104086221579197
8443 0.025054598777894392
8444 0.025076157354325182
8445 0.025032819990801577
8446 0.025009084543221347
8447 0.02499939808282623
8448 0.025005079494980792
8449 0.025025353466311066
8450 0.024976934351072153
8451 0.024945906060936107
8452 0.024944913831068956
8453 0.02492988089089338
8454 0.024933508006840115
8455 0.0249170039937714
8456 0.02491657479396266
8457 0.024901799001942206
8458 0.024898410468393226
8459 0.024894343093570782
8460 0.024904387377849974
8461 0.024932940799277253
8462 0.024888535007491948
8463 0.024854566945994742
8464 0.024828822459668977
8465 0.024835612780624197
8466 0.02484336900894783
8467 0.024810890626652732
8468 0.024821021359357473
8469 0.02478125339712

8905 0.02108775290934427
8906 0.02104789573535415
8907 0.021007849742534387
8908 0.020980244631502297
8909 0.0209877100293726
8910 0.02100348441558081
8911 0.02102417625492212
8912 0.02105108825901882
8913 0.021083802098621537
8914 0.0210201270581095
8915 0.02105031447624625
8916 0.021085982428047476
8917 0.02101710274454026
8918 0.021050115869389305
8919 0.021000147118229548
8920 0.02103349734428135
8921 0.02107237676546289
8922 0.021015844485702748
8923 0.020953025751521637
8924 0.02091367426471743
8925 0.02093775151328199
8926 0.020898605522493483
8927 0.020922675748063363
8928 0.020952371225423636
8929 0.02089455787382794
8930 0.020858544696357954
8931 0.020880135117902057
8932 0.020907346732438603
8933 0.020865094300772873
8934 0.02082968824434279
8935 0.020851103252483145
8936 0.020807690826472054
8937 0.02082625598249091
8938 0.020851340563358387
8939 0.02088249160387933
8940 0.02091929417069936
8941 0.020960383811401775
8942 0.02088046840801764
8943 0.020830548245204607
8944 0.

9404 0.017841223048695558
9405 0.017844766492241686
9406 0.01782836326988575
9407 0.017817988604536122
9408 0.01781671427088786
9409 0.01780594695148812
9410 0.017800390632100965
9411 0.017794489467511016
9412 0.017793517859578313
9413 0.0177832378738183
9414 0.0177817494834747
9415 0.01778550749079428
9416 0.017770701949765173
9417 0.017772865487132767
9418 0.017780087645392413
9419 0.017790637540744527
9420 0.017805840414313278
9421 0.017779660369712676
9422 0.01775488408701525
9423 0.017765178709433706
9424 0.01774445045177247
9425 0.01775388585362769
9426 0.01776817239654633
9427 0.01774316396903749
9428 0.017722428209892945
9429 0.017731977555623044
9430 0.017745551676128374
9431 0.017720550955279364
9432 0.01769979546558625
9433 0.017710286282641523
9434 0.01772474625783029
9435 0.017742940918067922
9436 0.01776379087816612
9437 0.01778812818347288
9438 0.017815754695544948
9439 0.017762420934964978
9440 0.01778842779514979
9441 0.017748581702665592
9442 0.017774940354243023
9443

9837 0.01577597228524725
9838 0.015782912161645353
9839 0.015766528255812835
9840 0.015772560379180935
9841 0.01578141123763954
9842 0.015793226202595434
9843 0.0157700458267812
9844 0.01578121408053474
9845 0.015795077204953926
9846 0.01581149242152632
9847 0.015784283091992788
9848 0.01575859187258285
9849 0.01573877912291695
9850 0.015748316609051478
9851 0.015727822807190983
9852 0.01571205380077261
9853 0.015700691945727197
9854 0.015703502312270023
9855 0.0156927051407712
9856 0.01569518739134648
9857 0.015683778084795368
9858 0.01567695729087734
9859 0.01567529116327103
9860 0.015667129044197783
9861 0.015666478187158644
9862 0.015668036377285973
9863 0.015656760666578114
9864 0.015659222644600693
9865 0.015647897975813656
9866 0.015641063968855076
9867 0.01563942085678337
9868 0.01564164423259967
9869 0.015645984440518425
9870 0.01565389430119017
9871 0.0156637380532244
9872 0.015676218616470425
9873 0.015691785469411863
9874 0.015663254784424378
9875 0.015639884544762273
9876 